In [ ]:
%pip install plotly

In [ ]:
import numpy as np

dataset_dir = "/mnt/local/argoverse/av1_traj_w_pose_egomotion"
results_dir = "/mnt/local/argoverse/av1_traj_w_pose_egomotion/_av1_traj_w_pose_egomotion_pred"
log_id = "033669d3-3d6b-3d3d-bd93-7985d86653ea"

data = np.load(f"{dataset_dir}/{log_id}.npz", allow_pickle=True)
results = np.load(f"{results_dir}/{log_id}_flow.npz", allow_pickle=True)

In [ ]:
import plotly.graph_objects as go
import numpy as np

def plot_point_cloud(points, color, size=1.8, **kwargs):
  return go.Scatter3d(
      x=points[:, 0],
      y=points[:, 1],
      z=points[:, 2],
      mode='markers',
      marker=dict(size=size, color=color),
      **kwargs,
  )

def _stack_with_none(a, b):
  ab = np.zeros((a.shape[0], 3))
  ab[:, 0] = a
  ab[:, 1] = b
  ab[:, 2] = None
  return ab.reshape((-1,))


def plot_flow_segments(a, b, name, **kwargs):
  return go.Scatter3d(
      x=_stack_with_none(a[:, 0], b[:, 0]),
      y=_stack_with_none(a[:, 1], b[:, 1]),
      z=_stack_with_none(a[:, 2], b[:, 2]),
      name=name,
      mode='lines',
      line=kwargs,
  )

In [ ]:
# Original data and inference results as is (no ego-motion compensation)
i = 0
p0_t0_ego = data['pcs'][i]
f0_t0_gt_ego = data['flows'][i]
f0_t0_pr_ego = results['flows'][i]
p1_t1_ego = data['pcs'][i+1]
p0_t1_gt_ego = p0_t0_ego + f0_t0_gt_ego
p0_t1_pr_ego = p0_t0_ego + f0_t0_pr_ego

fig = go.Figure(
    data=(
        plot_point_cloud(p0_t0_ego, color="red", size=1.3, name="#0"),
        plot_point_cloud(p1_t1_ego, color="blue", size=1.3, name="#1"),
        plot_flow_segments(p0_t0_ego, p0_t1_gt_ego, color="green", name="#0 -> #1 (GT)"),
        plot_flow_segments(p0_t0_ego, p0_t1_pr_ego, color="cyan", name="#0 -> #1 (PR)")
    ),
    layout=dict(
        width=1000,
        height=1000,
        scene=dict(aspectmode='data'),
    )
)
fig.show()

In [ ]:
# Post-processing to remove ego-motion
 
# Same as SE3.transform_point_cloud in https://github.com/argoverse/argoverse-api/blob/f886ac54fba9f06f8a7d109eb663c7f501b3aa8e/argoverse/utils/se3.py#L26C5-L36C64
def transform_point_cloud(city_from_ego: np.ndarray, point_cloud: np.ndarray) -> np.ndarray:
    rotation = city_from_ego[:3, :3]
    translation = city_from_ego[:3, 3]
    return point_cloud @ rotation.T + translation

city_from_ego_t0 = data['poses'][i]
city_from_ego_t1 = data['poses'][i+1]
p0_t0_city = transform_point_cloud(city_from_ego_t0, p0_t0_ego)
p1_t1_city = transform_point_cloud(city_from_ego_t1, p1_t1_ego)
# MBNSF uses ego@t1 for the flow
p0_t1_gt_city = transform_point_cloud(city_from_ego_t1, p0_t1_gt_ego)
p0_t1_pr_city = transform_point_cloud(city_from_ego_t1, p0_t1_pr_ego)

fig = go.Figure(
    data=(
        plot_point_cloud(p0_t0_city, color="red", size=1.3, name="#0"),
        plot_point_cloud(p1_t1_city, color="blue", size=1.3, name="#1"),
        plot_flow_segments(p0_t0_city, p0_t1_gt_city, color="green", name="#0 -> #1 (GT)"),
        plot_flow_segments(p0_t0_city, p0_t1_pr_city, color="cyan", name="#0 -> #1 (PR)")
    ),
    layout=dict(
        width=1000,
        height=1000,
        scene=dict(aspectmode='data'),
    )
)
fig.show()